## ET5

In [1]:
import json
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer, pipeline
from tqdm import tqdm
import pandas as pd
import re
from difflib import SequenceMatcher

import warnings
warnings.filterwarnings('ignore')

def load_data(file_path):
    with open(file_path, 'r') as f:
        json_data = json.load(f)
        
    data = []
    for item in json_data['data']:
        if 'annotation' in item:
            annotation = item['annotation']
            
            if 'err_sentence' in annotation and 'cor_sentence' in annotation:
                data.append({
                    'err_sentence': annotation['err_sentence'],
                    'cor_sentence': annotation['cor_sentence']
                })
                
    df = pd.DataFrame(data)
    print(df.info())
    # None 값을 빈 문자열로 대체
    df = df.fillna('')
    return df

def apply_spelling_correction(text):
    if not isinstance(text, str) or not text.strip():
        return ""
    
    try:
        # 최대 토큰 길이 설정
        max_token_length = 128 - 2  # CLS와 SEP 토큰을 위한 자리
        
        # 공백 제거된 텍스트 준비
        org_text = text.replace(" ", "")
        corrected_sentences = []
        
        # 입력 텍스트가 비어있는 경우 처리
        if not org_text:
            return ""
        
        # 청크 단위로 처리
        for start_idx in range(0, len(org_text), max_token_length):
            chunk = org_text[start_idx:start_idx + max_token_length]
            
            # 각 청크에 대해 모델 처리
            input_encoding = tokenizer("맞춤법을 고쳐주세요: " + chunk, return_tensors="pt")
            input_ids = input_encoding.input_ids.to(device)
            attention_mask = input_encoding.attention_mask.to(device)
            
            output_encoding = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_length=128,
                num_beams=5,
                early_stopping=True,
            )
            
            chunk_result = tokenizer.decode(output_encoding[0], skip_special_tokens=True)
            if chunk_result.strip():  # 빈 문자열이 아닌 경우에만 추가
                corrected_sentences.append(chunk_result.strip())
        
        # 모든 청크를 다시 합치기
        final_text = " ".join(corrected_sentences)
        return final_text.strip() or ""  # 빈 문자열이면 "" 반환
        
    except Exception as e:
        print(f"Error processing text: {text}")
        print(f"Error message: {str(e)}")
        return ""

def process_dataframe(df, text_column):
    # None 값을 빈 문자열로 대체
    df[text_column] = df[text_column].fillna('')
    
    tqdm.pandas(desc="Processing spelling correction")
    df["res_sentence"] = df[text_column].progress_apply(apply_spelling_correction)
    
    # None 값을 빈 문자열로 대체
    df["res_sentence"] = df["res_sentence"].fillna('')
    
    return df

def calculate_spacing_similarity(row):
    """
    두 문장 간의 유사도를 계산하는 함수 - 단어별 일치율 기반
    """
    # 빈 문자열 처리
    if not row['res_sentence'] or not row['cor_sentence']:
        return 0.0

    # 문장을 띄어쓰기로 분할하여 각 단어 비교
    res_words = row['res_sentence'].split()
    cor_words = row['cor_sentence'].split()

    # 비교할 단어 쌍의 수
    max_len = max(len(res_words), len(cor_words))
    if max_len == 0:
        return 1.0  # 둘 다 빈 문장일 경우 100% 일치로 처리

    # 단어별 일치율 계산
    match_count = sum(1 for res_word, cor_word in zip(res_words, cor_words) if res_word == cor_word)
    word_accuracy = match_count / max_len

    # 전체 문자열의 유사도 계산 (띄어쓰기 포함)
    similarity = SequenceMatcher(None, row['res_sentence'], row['cor_sentence']).ratio()

    # 최종 비율 (가중치는 조정 가능)
    final_ratio = (similarity + word_accuracy) / 2

    return round(final_ratio, 3)

def add_comparison_columns(df):
    # None 값을 빈 문자열로 대체
    df['res_sentence'] = df['res_sentence'].fillna('')
    df['cor_sentence'] = df['cor_sentence'].fillna('')
    
    # check 컬럼 추가 (완전히 같으면 1, 다르면 0)
    df['check'] = (df['res_sentence'] == df['cor_sentence']).astype(int)
    
    # ratio 컬럼 추가
    df['ratio'] = df.apply(calculate_spacing_similarity, axis=1)
    
    return df

def text_clean(text):
    if not isinstance(text, str):
        return ""
        
    pattern = '[^\w\s]'         # 특수기호제거
    text = re.sub(pattern, '', text)
    return text 

# 모델 정의 (전역 변수로 유지)
model = T5ForConditionalGeneration.from_pretrained('j5ng/et5-typos-corrector')
tokenizer = T5Tokenizer.from_pretrained('j5ng/et5-typos-corrector')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

if __name__ == "__main__":
    df = load_data('/home/yjtech2/Desktop/yurim/LLM/DATA/맞춤법오류_자유게시판.json')
    processed_df = process_dataframe(df, "err_sentence")
    print('처리 완료!')

    processed_df = processed_df[['err_sentence', 'res_sentence', 'cor_sentence']]
    
    processed_df['res_sentence'] = processed_df['res_sentence'].apply(text_clean)
    processed_df['cor_sentence'] = processed_df['cor_sentence'].apply(text_clean)
    
    result_df = add_comparison_columns(processed_df)
    
    # 결과 출력
    print("비교 완료")
    print('정확도: ', (len(result_df[result_df['check'] == 1]) / len(result_df)))
    print('비율 평균: ', sum(result_df['ratio']) / len(result_df))

/home/yjtech2/Desktop/yurim/anaconda3/envs/venv-llm/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-11-14 14:59:27.152165: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-14 14:59:27.178123: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-14 14:59:27.521660: W tensorflow/compiler/tf2tensorrt/utils/py_utils.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14400 entries, 0 to 14399
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   err_sentence  14400 non-null  object
 1   cor_sentence  14400 non-null  object
dtypes: object(2)
memory usage: 225.1+ KB
None


Processing spelling correction: 100%|██████████| 14400/14400 [19:24<00:00, 12.36it/s]


처리 완료!
비교 완료
정확도:  0.265625
비율 평균:  0.7866309027777771


In [2]:
result_df

,err_sentence,res_sentence,cor_sentence,check,ratio
0,오늘도 다너를 외우러 와따요,오늘도 너를 외우러 왔어요,오늘도 단어를 외우러 왔어요,0,0.823
1,오늘은 추석 연휴동안 놀고 먹으며 학회 레포트를 쓰는 영상 준비해보았습니디,오늘은 추석 연휴 동안 놀고 먹으며 학회리포트를 쓰는 영상 준비해 보았습니다,오늘은 추석 연휴 동안 놀고먹으며 학회 리포트를 쓰는 영상 준비해 보았습니다,0,0.852
2,요즘 새로 살까말까 고민중인데 대학원 가면 새로 살것 같기도 하구 그러네요,요즘 새로 살까 말까 고민 중인데 대학원 가면 새로 살 것 같기도 하고 그러네요,요즘 새로 살까 말까 고민 중인데 대학원 가면 새로 살 것 같기도 하고 그러네요,1,1.000
3,요즘 영어 문제르 안 풀어서 실력이 내려가고 있는거 가타요.,요즘 영어 문제를 안 풀어서 실력이 내려가고 있는 거 같아요,요즘 영어 문제를 안 풀어서 실력이 내려가고 있는 거 같아요,1,1.000
4,원래 좋아하는 건 잘 찾아 외우는 능력이 잇서서 말이죠,원래 좋아하는 건 잘 찾아 외우는 능력이 있어서 말이죠,원래 좋아하는 것은 잘 찾아 외우는 능력이 있어서 말이죠,0,0.920
...,...,...,...,...,...
14395,그 사람 이야기하는거 아님,그 사람 이야기하는 거 아님,그 사람 이야기하는 거 아님,1,1.000
14396,그러다가 저 연세대갈사람 이 생사람 잡으면서 동일인물이라고 해서 기세등등해짐,그러다가 저 연세대 갈 사람이 생사람 잡으면서 동일 인물이라고 해서 기세 등등해짐,그러다가 저 연세대 갈 사람이 생사람 잡으면서 동일 인물이라고 해서 기세등등해짐,0,0.911
14397,그런것들은 수능 공부를 병행하면서 준비해주시면 됩니다.,그런 것들은 수능 공부를 병행하면서 준비해 주시면 됩니다,그런 것들은 수능 공부를 병행하면서 준비해 주시면 됩니다,1,1.000
14398,그리고 오늘은 유독 많은 일이 있었던거 같네요,그리고 오늘은 유독 많은 일이 있었던 거 같네요,그리고 오늘은 유독 많은 일이 있었던 거 같네요,1,1.000
